In [5]:
import pandas as pd
clean = "../../data/clean/"
with open("../../data/raw/LgaRankings_27_Offences.xlsx", "rb") as f:
    df_dict = pd.read_excel(f, sheet_name=None, encoding='latin-1', skiprows=0, header=[4,5], na_values=['nc'], skip_footer=7)


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/pandas/util/_decorators.py:177: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


ValueError: Duplicated level name: "Local Government Area", assigned to level 1, is already used for level 0.

In [3]:
uns = pd.DataFrame(df_dict['Assault - domestic violence'].unstack())

uns.index = uns.index.rename(["Year","Type", "LGA"])
uns.columns = ['Assault - domestic violence']
uns

ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
cleaned_list = []

for key in df_dict:
    unstacked = pd.DataFrame(df_dict[key].unstack())

    unstacked.index = unstacked.index.rename(["Year","Type", "LGA"])
    unstacked.columns = [key]
    cleaned_list.append(unstacked)
    
final_df = cleaned_list[0]
for i in range(1,len(cleaned_list)):
    final_df = pd.merge(final_df, cleaned_list[i], how='inner', left_index=True, right_index=True)


In [24]:
output_df = final_df.reset_index()
output_df = output_df[output_df["Type"] == 'Rate per 100,000 population']
output_df = output_df.dropna()
output_df.loc[output_df["LGA"] == "Greater Hume Shire", "LGA"] = "Greater Hume"
output_df = output_df.loc[output_df["LGA"] != "Lord Howe Island"]

NameError: name 'final_df' is not defined

In [ ]:
output_df.columns

In [26]:
nonzeroes = output_df.astype(bool).sum(axis=0).sort_values(ascending=False)
nonzeroes = nonzeroes[nonzeroes> 540]
nonzeroes = nonzeroes.drop(['Type','LGA', 'Year'])

25

In [ ]:
output_df.corr()["Other theft"].sort_values()

In [ ]:
output_df.to_csv("cleaned_target.csv")

In [51]:
output_df = pd.read_csv(clean+"cleaned_target.csv")
import numpy as np
#output_df.iloc[:,4:] = np.log(output_df.iloc[:,4:])
output_df

,Unnamed: 0,Year,Type,LGA,Assault - domestic violence,Assault - non-domestic violence,Sexual Offences,Robbery,Break and enter dwelling,Break and enter non-dwelling,...,Possession use of cannabis,Prohibited weapons offences,Trespass,Offensive conduct,Offensive language,Liquor offences,Breach AVO,Breach bail conditions,Resist or hinder officer,Transport regulatory offences
0,130,2013,"Rate per 100,000 population",Albury,548.1193,782.4602,218.4534,21.8453,641.4585,468.6817,...,162.8470,254.2003,208.5237,303.8488,129.0861,389.2442,256.1862,1036.6604,148.9455,19.8594
1,131,2013,"Rate per 100,000 population",Armidale Regional,663.4881,796.8526,266.7289,33.3411,916.8806,480.1120,...,413.4298,150.0350,226.7196,273.3971,146.7009,416.7639,316.7406,283.3995,140.0327,20.0047
2,132,2013,"Rate per 100,000 population",Ballina,340.7608,454.3477,198.1729,12.0837,408.4296,217.5069,...,210.2567,79.7525,82.1693,60.4186,36.2511,116.0037,222.3404,174.0055,55.5851,2.4167
3,134,2013,"Rate per 100,000 population",Bathurst Regional,477.9646,696.1126,178.9303,36.7665,799.0588,343.1541,...,286.7788,191.1858,200.9902,78.4352,36.7665,134.8105,218.1479,387.2739,105.3973,7.3533
4,135,2013,"Rate per 100,000 population",Bayside,254.7872,263.3681,79.8685,59.4063,290.4310,84.4890,...,250.8267,93.7300,56.7661,23.7625,30.3632,34.9837,110.8918,188.7801,37.6240,1267.9952
5,136,2013,"Rate per 100,000 population",Bega Valley,310.9676,490.3720,146.5136,8.9702,149.5036,194.3547,...,307.9775,134.5533,125.5831,182.3945,131.5632,340.8683,182.3945,122.5930,89.7022,0.0000
6,137,2013,"Rate per 100,000 population",Bellingen,193.7684,348.7831,139.5133,15.5015,286.7772,341.0324,...,248.0236,85.2581,85.2581,38.7537,54.2552,131.7625,224.7714,85.2581,77.5074,0.0000
7,138,2013,"Rate per 100,000 population",Berrigan,284.9003,249.2877,130.5793,0.0000,367.9962,308.6420,...,356.1254,296.7711,249.2877,130.5793,83.0959,284.9003,178.0627,106.8376,71.2251,0.0000
8,139,2013,"Rate per 100,000 population",Blacktown,574.6330,543.5301,160.7494,112.7094,525.3611,122.8717,...,206.0179,117.9445,94.5404,24.9439,33.2585,60.3580,222.6472,641.1500,58.2024,2022.3017
9,140,2013,"Rate per 100,000 population",Bland,430.9630,430.9630,281.7835,16.5755,348.0855,281.7835,...,165.7550,397.8120,198.9060,198.9060,66.3020,795.6241,281.7835,215.4815,165.7550,0.0000


In [49]:
import numpy as np
variance = output_df.var()
corr = output_df.loc[:,nonzeroes.index].corr()
while len(corr) > 6:
    maxvals = []
    argmaxs = []
    for i,x in corr.iterrows():
        argmax = 0
        maxval = 0

        for j, y in enumerate(x[1:]):
            if y > maxval and y != 1.0:
                maxval = y
                argmax = j
        #print(i, maxval, x.index[argmax+1])
        maxvals.append(maxval)
        argmaxs.append(argmax)

    #the most correlated - drop the one with less variance
    mostcorridx = np.argmax(maxvals)
    mostcorr = corr.index[mostcorridx]
    mostcorr_other = corr.index[argmaxs[mostcorridx]+1]
    if variance[mostcorr] > variance[mostcorr_other]:
        corr = corr.drop(mostcorr_other)
        corr = corr.drop(mostcorr_other, axis=1)
        print("Dropping "+mostcorr_other)
    else:
        corr = corr.drop(mostcorr)
        corr = corr.drop(mostcorr, axis=1)      
        print("Dropping "+mostcorr)

Dropping Assault - domestic violence
Dropping Malicious damage to property
Dropping Harassment threatening
Dropping Offensive conduct
Dropping Resist or hinder officer
Dropping Assault - non-domestic violence
Dropping Steal from dwelling
Dropping Break and enter dwelling
Dropping Breach AVO
Dropping Trespass
Dropping Steal from motor vehicle
Dropping Steal from retail store
Dropping Break and enter non-dwelling
Dropping Breach bail conditions
Dropping Sexual Offences
Dropping Prohibited weapons offences
Dropping Liquor offences
Dropping Possession use of cannabis
Dropping Other theft


In [50]:
corr

,Fraud,Unnamed: 0,Motor vehicle theft,Arson,Offensive language,Receiving stolen goods
Fraud,1.000000,0.119578,0.232281,-0.186014,-0.085199,0.465389
Unnamed: 0,0.119578,1.000000,-0.041272,-0.067471,-0.092245,0.069467
Motor vehicle theft,0.232281,-0.041272,1.000000,0.593701,0.375744,0.512491
Arson,-0.186014,-0.067471,0.593701,1.000000,0.533780,0.282567
Offensive language,-0.085199,-0.092245,0.375744,0.533780,1.000000,0.473039
Receiving stolen goods,0.465389,0.069467,0.512491,0.282567,0.473039,1.000000


In [28]:
np.sum(output_df == 0)

Unnamed: 0                           0
Year                                 0
Type                                 0
LGA                                  0
Assault - domestic violence          0
Assault - non-domestic violence      0
Sexual Offences                      0
Robbery                            124
Break and enter dwelling             0
Break and enter non-dwelling         0
Motor vehicle theft                  3
Steal from motor vehicle             1
Steal from retail store             13
Steal from dwelling                  0
Steal from person                   94
Fraud                                0
Malicious damage to property         0
Harassment threatening               0
Receiving stolen goods              32
Other theft                          0
Arson                                4
Possession use of cannabis           2
Prohibited weapons offences          0
Trespass                             0
Offensive conduct                    6
Offensive language       

In [ ]:
corr = output_df.iloc[:,1:].corr()

In [ ]:
corr

In [52]:
# just keep chosen lgas and chosen targets
output_df = output_df[output_df["Year"] == 2016]
output_df = output_df[['LGA', 'Assault - domestic violence', 'Assault - non-domestic violence', 'Fraud', 'Sexual Offences', 'Malicious damage to property', 'Prohibited weapons offences',]]
output_df

,LGA,Assault - domestic violence,Assault - non-domestic violence,Fraud,Sexual Offences,Malicious damage to property,Prohibited weapons offences
362,Albury,544.4263,563.5963,525.2564,216.6203,1315.0580,316.3040
363,Armidale Regional,501.4681,653.2282,841.2787,221.0419,2217.0169,287.0245
364,Ballina,255.7125,321.4001,504.3870,154.8351,795.2893,152.4891
365,Bathurst Regional,438.7931,462.3841,686.4989,200.5237,1184.2695,226.4738
366,Bayside,315.9874,274.1388,728.4085,85.5167,584.6676,108.5638
367,Bega Valley,312.2606,341.7192,318.1524,153.1845,810.1102,153.1845
368,Bellingen,279.2213,263.7090,248.1967,116.3422,604.9794,93.0738
369,Berrigan,174.9883,139.9907,163.3224,81.6612,699.9533,279.9813
370,Blacktown,594.3767,453.1188,821.0799,163.9858,980.7504,148.7380
371,Bland,348.2010,315.0390,281.8770,215.5530,746.1449,381.3630


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,Unnamed: 0,Year,Type,LGA,Assault - domestic violence,Assault - non-domestic violence,Sexual Offences,Robbery,Break and enter dwelling,Break and enter non-dwelling,...,Possession use of cannabis,Prohibited weapons offences,Trespass,Offensive conduct,Offensive language,Liquor offences,Breach AVO,Breach bail conditions,Resist or hinder officer,Transport regulatory offences
0,130,NaN,NaN,NaN,6.306493,6.662443,5.386573,3.083986,6.463744,6.149924,...,5.092811,5.538123,5.340053,5.716530,4.860480,5.964207,5.545905,6.943760,5.003580,2.988677
1,131,NaN,NaN,NaN,6.497511,6.680670,5.586233,3.506791,6.820977,6.174019,...,6.024488,5.010869,5.423714,5.610925,4.988396,6.032520,5.758083,5.646858,4.941876,2.995967
2,132,NaN,NaN,NaN,5.831181,6.118863,5.289140,2.491857,6.012320,5.382231,...,5.348329,4.378928,4.408782,4.101297,3.590470,4.753622,5.404210,5.159087,4.017915,0.882403
3,134,NaN,NaN,NaN,6.169537,6.545511,5.186996,3.604587,6.683435,5.838180,...,5.658711,5.253246,5.303256,4.362273,3.604587,4.903870,5.385173,5.959132,4.657737,1.995149
4,135,NaN,NaN,NaN,5.540429,5.573553,4.380382,4.084400,5.671366,4.436621,...,5.524762,4.540418,4.038939,3.168109,3.413231,3.554882,4.708555,5.240583,3.627642,7.145192
5,136,NaN,NaN,NaN,5.739689,6.195164,4.987118,2.193908,5.007320,5.269685,...,5.730027,4.901960,4.832968,5.206172,4.879487,5.831496,5.206172,4.808870,4.496495,-inf
6,137,NaN,NaN,NaN,5.266664,5.854450,4.938160,2.740937,5.658706,5.831977,...,5.513524,4.445683,4.445683,3.657226,3.993699,4.881001,5.415084,4.445683,4.350373,-inf
7,138,NaN,NaN,NaN,5.652139,5.518608,4.871981,-inf,5.908073,5.732182,...,5.875283,5.692961,5.518608,4.871981,4.419995,5.652139,5.182136,4.671310,4.265845,-inf
8,139,NaN,NaN,NaN,6.353732,6.298085,5.079847,4.724813,6.264086,4.811141,...,5.327963,4.770214,4.549027,3.216629,3.504310,4.100293,5.405588,6.463263,4.063927,7.611992
9,140,NaN,NaN,NaN,6.066022,6.066022,5.641139,2.807926,5.852448,5.641139,...,5.110511,5.985980,5.292832,5.292832,4.194220,6.679127,5.641139,5.372875,5.110511,-inf


In [53]:
output_df.to_csv(clean+"filtered_nonlog_target.csv")